In [4]:
%%capture --no-stderr
%pip install -U langchain_community langchain_openai langchain_fireworks langgraph wikipedia duckduckgo-search tavily-python

  You can safely remove it manually.


In [5]:
from langchain_community.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain_core.tools import tool

In [6]:
# DDG
search_engine = DuckDuckGoSearchAPIWrapper()

In [51]:
results = DuckDuckGoSearchAPIWrapper()._ddgs_text("aa")

In [52]:
results


[{'title': 'Alcoholics Anonymous - Wikipedia',
  'href': 'https://en.wikipedia.org/wiki/Alcoholics_Anonymous',
  'body': 'Alcoholics Anonymous (AA) is a global, peer-led mutual-aid fellowship focused on an abstinence-based recovery model from alcoholism through its spiritually-inclined twelve-step program. [1] The organization adheres to Twelve Traditions that emphasize anonymity, the absence of a hierarchical structure, and principles of being free to all, non-promotional, non-professional, unaffiliated, non ...'},
 {'title': 'International Convention - Alcoholics Anonymous',
  'href': 'https://www.aa.org/international-convention',
  'body': "Areas and districts may wish to link their Web sites directly to the 2025 International Convention Web page. This notice represents a limited authorization to all Area and District Web sites to use the Service Mark, 2025 International Convention logo, ONLY as an icon link on their websites to G.S.O.'s A.A. 2025 International Convention web page."

# Setup

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Define graph state

In [2]:
from typing import List
from typing_extensions import TypedDict


class ReWOO(TypedDict):
    task: str
    plan_string: str
    steps: List
    results: dict
    result: str

# Planner

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")


In [60]:
prompt = """For the following task, make plans that can solve the problem step by step. For each plan, indicate \
which external tool together with tool input to retrieve evidence. You can store the evidence into a \
variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)

Tools can be one of the following:
(1) Duckduckgo[input]: Worker that searches results from Duckduckgo. Useful when you need to find short
and succinct answers about a specific topic. The input should be a search query.
(2) LLM[input]: A pretrained LLM like yourself. Useful when you need to act with general
world knowledge and common sense. Prioritize it when you are confident in solving the problem
yourself. Input can be any instruction.

For example,
Task: Thomas, Toby, and Rebecca worked a total of 157 hours in one week. Thomas worked x
hours. Toby worked 10 hours less than twice what Thomas worked, and Rebecca worked 8 hours
less than Toby. How many hours did Rebecca work?
Plan: Given Thomas worked x hours, translate the problem into algebraic expressions and solve
with Wolfram Alpha. #E1 = WolframAlpha[Solve x + (2x − 10) + ((2x − 10) − 8) = 157]
Plan: Find out the number of hours Thomas worked. #E2 = LLM[What is x, given #E1]
Plan: Calculate the number of hours Rebecca worked. #E3 = Calculator[(2 ∗ #E2 − 10) − 8]

Begin! 
Describe your plans with rich details. Each Plan should be followed by only one #E.

Task: {task}"""

In [61]:
task = "what is the exact hometown of the 2024 mens australian open winner"


In [62]:
result = model.invoke(prompt.format(task=task))


In [63]:
prompt.format(task=task)


'For the following task, make plans that can solve the problem step by step. For each plan, indicate which external tool together with tool input to retrieve evidence. You can store the evidence into a variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)\n\nTools can be one of the following:\n(1) Duckduckgo[input]: Worker that searches results from Duckduckgo. Useful when you need to find short\nand succinct answers about a specific topic. The input should be a search query.\n(2) LLM[input]: A pretrained LLM like yourself. Useful when you need to act with general\nworld knowledge and common sense. Prioritize it when you are confident in solving the problem\nyourself. Input can be any instruction.\n\nFor example,\nTask: Thomas, Toby, and Rebecca worked a total of 157 hours in one week. Thomas worked x\nhours. Toby worked 10 hours less than twice what Thomas worked, and Rebecca worked 8 hours\nless than Toby. How many hours did Rebecca work?\nPlan: Given Thoma

In [65]:
print(result.content)


Plan: Identify the winner of the 2024 Men's Australian Open. A search engine can quickly provide this information as it is a factual question based on a recent event. #E1 = Duckduckgo["2024 Men's Australian Open winner"]

Plan: After identifying the winner, find out the exact hometown of the player. This will likely require biographical information about the player, which can be found by searching for details about their personal background. #E2 = Duckduckgo["hometown of #E1 winner"]


### Planner Node

In [66]:
import re

from langchain_core.prompts import ChatPromptTemplate


In [67]:
# Regex to match expressions of the form E#... = ...[...]
regex_pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\[([^\]]+)\]"
prompt_template = ChatPromptTemplate.from_messages([("user", prompt)])
planner = prompt_template | model

In [68]:
def get_plan(state: ReWOO): # state by a TypedDict
    task = state["task"]
    result = planner.invoke({"task": task})
    # Find all matches in the sample text
    matches = re.findall(regex_pattern, result.content)
    return {"steps": matches, "plan_string": result.content}

# Executor

In [69]:
# DDG
search = DuckDuckGoSearchAPIWrapper()

In [70]:
def _get_current_task(state: ReWOO):
    if "results" not in state or state["results"] is None:
        return 1
    if len(state["results"]) == len(state["steps"]):
        return None
    else:
        return len(state["results"]) + 1

In [71]:
# DuckDuckGoSearchAPIWrapper()._ddgs_text("aa")


In [72]:
def tool_execution(state: ReWOO):
    """Worker node that executes the tools of a given plan."""
    _step = _get_current_task(state)
    _, step_name, tool, tool_input = state["steps"][_step - 1]
    _results = (state["results"] or {}) if "results" in state else {}
    for k, v in _results.items():
        tool_input = tool_input.replace(k, v) # replace key with value
    if tool == "Duckduckgo":
        # result = search._ddgs_text(tool_input)
        result = search.run(tool_input)

    elif tool == "LLM":
        result = model.invoke(tool_input)
    else:
        raise ValueError
    _results[step_name] = str(result)
    return {"results": _results}

# Solver


In [73]:
solve_prompt = """Solve the following task or problem. To solve the problem, we have made step-by-step Plan and \
retrieved corresponding Evidence to each Plan. Use them with caution since long evidence might \
contain irrelevant information.

{plan}

Now solve the question or task according to provided Evidence above. Respond with the answer
directly with no extra words.

Task: {task}
Response:"""


def solve(state: ReWOO):
    plan = ""
    for _plan, step_name, tool, tool_input in state["steps"]:
        _results = (state["results"] or {}) if "results" in state else {}
        for k, v in _results.items():
            tool_input = tool_input.replace(k, v)
            step_name = step_name.replace(k, v)
        plan += f"Plan: {_plan}\n{step_name} = {tool}[{tool_input}]"
    prompt = solve_prompt.format(plan=plan, task=state["task"])
    result = model.invoke(prompt)
    return {"result": result.content}

# Define Graph

In [74]:
def _route(state):
    _step = _get_current_task(state)
    if _step is None:
        # We have executed all tasks
        return "solve"
    else:
        # We are still executing tasks, loop back to the "tool" node
        return "tool"

In [75]:
from langgraph.graph import END, StateGraph, START

graph = StateGraph(ReWOO)
graph.add_node("plan", get_plan)
graph.add_node("tool", tool_execution)
graph.add_node("solve", solve)
graph.add_edge("plan", "tool")
graph.add_edge("solve", END)
graph.add_conditional_edges("tool", _route)
graph.add_edge(START, "plan")

app = graph.compile()

In [76]:
for s in app.stream({"task": task}):
    print(s)
    print("---")
    

{'plan': {'steps': [("To identify the 2024 Men's Australian Open winner, I will search for the results of the tournament. ", '#E1', 'Duckduckgo', '"2024 Men\'s Australian Open winner"'), ('Once the winner is identified, I will search for the specific hometown of the winner. ', '#E2', 'Duckduckgo', '"hometown of [winner\'s name')], 'plan_string': 'Plan: To identify the 2024 Men\'s Australian Open winner, I will search for the results of the tournament. #E1 = Duckduckgo["2024 Men\'s Australian Open winner"]\n\nPlan: Once the winner is identified, I will search for the specific hometown of the winner. #E2 = Duckduckgo["hometown of [winner\'s name]"]\n\nNote: Replace [winner\'s name] with the actual name obtained from #E1.'}}
---


ValueError: 

In [36]:
# Print out the final result
print(s["solve"]["result"])

I'm sorry, but I can't provide real-time information or future events beyond my last update in October 2023.
